In [ ]:
from matplotlib.colors import SymLogNorm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates.sky_coordinate import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo
from astropy.wcs import WCS
from minot import model

# Modify plotting parameters
dict_base = {'font.size':        16,
             'legend.fontsize':  16,
             'xtick.labelsize':  16,
             'ytick.labelsize':  16,
             'axes.labelsize':   16,
             'axes.titlesize':   16,
             'figure.titlesize': 16,    
             'figure.figsize':[8.0, 6.0],
             'figure.subplot.right':0.97,
             'figure.subplot.left':0.15,
             'font.family':'serif',
             'figure.facecolor': 'white',
             'legend.frameon': True}
plt.rcParams.update(dict_base)

In [ ]:
clust = model.Cluster(name='A2151CB', 
                      redshift=0.0368, M500=14.32e13*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/testA2151CB')

clust_two = model.Cluster(name='A2199', 
                      redshift=0.0368, M500=0.9e13*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/testA2151CF')

def gNFW_normalization(redshift, M500, cosmo=cosmo):
    """
    Compute a gNFW model electronic pressure normalization based on mass and redshift.
    See Arnaud et al. A&A 517, A92 (2010). This does not account to the P0 
    term, which should be multiplied to get the overall normalization.
    Parameters
    ----------
    - redshift: redshift of the cluster
    - M500 (Msun): the mass within R500 of the cluster 
    Outputs
    --------
    - Pnorm (keV/cm^-3): the electron pressure normalization
    """

    E_z = cosmo.efunc(redshift)
    h70 = cosmo.H0.value/70.0

    F_M = (M500/3e14*h70)**0.12
    P500 = 1.65e-3 * E_z**(8.0/3.0) * (M500/3e14*h70)**(2.0/3.0) * h70**2
    
    Pnorm = P500 * F_M
    
    return Pnorm

In [ ]:
#----- Some parameters that can be set at when defining the cluster
clust.coord = SkyCoord("241.14d  17.72d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust.density_gas_model = {'name':'beta', 'n_0':8.51e-3*u.cm**-3, 'beta':0.38, 'r_c':19.35*u.kpc}



#UPP Planck P0 From Tiwari
clust.pressure_gas_model = {'name':'GNFW', 'P_0':0.009*u.keV/u.cm**3, 'c500':1.81, 'a':1.24, 'b':4.13, 'c':0.31}



# Set the cosmic ray density profile to the same shape as the thermal gas density
clust.set_density_crp_isodens_scal_param()
# Cosmic ray proton spectrum
clust.spectrum_crp_model = {'name':'ExponentialCutoffPowerLaw', 'Index':2., 'CutoffEnergy':10e5*u.TeV}
# Cosmic ray proton normalization
clust.X_crp_E = {'X':0.1, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm
#Set magnetic field profile
clust.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)
clust.abundance = 0.43
clust.Epmax = 10e6*u.TeV
clust.Eemax = 10e6*u.TeV

#__________________________________________________________________________________________________



#----- Some parameters that can be set at when defining the cluster
clust_two.coord = SkyCoord("241.28d  17.72d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust_two.density_gas_model = {'name':'beta', 'n_0':3.87e-3*u.cm**-3, 'beta':0.28, 'r_c':10.43*u.kpc}





# Set the cosmic ray density profile to the same shape as the thermal gas density
clust_two.set_density_crp_isodens_scal_param()
# Cosmic ray proton spectrum
clust_two.spectrum_crp_model = {'name':'ExponentialCutoffPowerLaw', 'Index':2., 'CutoffEnergy':10e5*u.TeV}
# Cosmic ray proton normalization
clust_two.X_crp_E = {'X':0.05, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm
#Set magnetic field profile
clust_two.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)
clust_two.abundance = 0.13
clust_two.Epmax = 10e6*u.TeV
clust_two.Eemax = 10e6*u.TeV

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')

# разом з СТА чутливістю
plt.figure(0,figsize=(10,8))
bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
        1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA North sensivity 50h')

#LHAASO sensitivity
bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data, color='r', ls='-.', lw='2', alpha=0.5, label = 'LHAASO sensivity')


sum = s1 + s2
plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume A2151CB')
plt.loglog(e1, (e1**2*sum).to('GeV cm-2 s-1'), '-', label='Total volume of two subclumps')
plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='Total volume A2151CF')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
r = [15, 45, 75, 105, 135, 165]
P = [14e-3, 9.26e-3, 5.6e-3, 6.14e-3, 3.75e-3, 2.6e-3]
r1, p1 = clust.get_pressure_gas_profile()


plt.figure(0,figsize=(10,8))
plt.loglog(r1,p1, '-', label = 'Pressure profile MINOT')
plt.loglog(r,P, '-', label = 'Pressure profile Tiwari')
plt.xlabel('Radius, kpc')
plt.ylabel('Pressure, keV/cm^3')
plt.gca().set_ylim(bottom= 1e-3)
plt.gca().set_xlim(left= 10)
plt.legend()



In [ ]:
0.0047 / gNFW_normalization(0.038, 14.32e13)

In [ ]:
r = [15, 45, 75, 105, 135, 195]
P = [14e-3, 9.26e-3, 5.6e-3, 6.14e-3, 3.75e-3, 2.6e-3]
r1, p1 = clust.get_pressure_gas_profile()
plt.figure(figsize = (12,9))
ax = plt.subplot()
ax.set_xscale("log", nonposx='clip')
ax.set_yscale("log", nonposy='clip')
ax.set(title='A2151CB')
yer = [0.0011200000000000001, 0.0012964, 0.0008399999999999999, 0.001535, 0.0012, 0.000312 ]
xer = [[15,15,15,15,15,35],[15,15,15,15,15,35]]
ax.loglog(r1,p1, '-', label = 'Theoretical pressure profile MINOT')
ax.errorbar(r, P, xerr=xer, yerr=yer, fmt = '.k', label = 'XMM-Newton data')

# ylim must be set after errorbar to allow errorbar to autoscale limits
ax.set_ylim(bottom= 1e-3)
ax.set_xlim(left = 7)
plt.xlabel('Radius, kpc')
plt.ylabel('Pressure, keV/cm^3')
plt.legend()
#fig.tight_layout()

plt.show()

In [ ]:
P

In [ ]:
0.0026*0.12

In [ ]:
r = [15, 45, 75, 105, 135, 165]
P = [14e-3, 9.26e-3, 5.6e-3, 6.14e-3, 3.75e-3, 2.6e-3]
r1, p1 = clust.get_pressure_gas_profile()
plt.figure(figsize = (12,9))
ax = plt.subplot()
ax.set_xscale("log", nonposx='clip')
ax.set_yscale("log", nonposy='clip')
ax.set(title='A2151CB')
yer = [0.0011200000000000001, 0.0012964, 0.0008399999999999999, 0.001535, 0.0012, 0.000312 ]
xer = [[15,15,15,15,15,15],[15,15,15,15,15,75]]

ax.loglog(r1,p1, '-', c = 'r', label = 'Theoretical pressure profile MINOT')
ax.errorbar(r, P, xerr=xer, yerr=yer, fmt = '.k', label = 'XMM-Newton data')

# ylim must be set after errorbar to allow errorbar to autoscale limits
ax.set_ylim(bottom= 1e-3)
ax.set_xlim(left = 7, right = 10000)
plt.xlabel('Radius, kpc')
plt.ylabel('Pressure, keV/cm^3')
plt.legend()
#fig.tight_layout()

plt.show()

In [ ]:
clust.density_gas_model = {'name':'beta', 'n_0':10.5e-3*u.cm**-3, 'beta':0.32, 'r_c':19.35*u.kpc}
#clust.density_gas_model = {'name':'doublebeta', 'n_01':10.5e-3*u.cm**-3, 'beta1':0.32, 'r_c1':11.35*u.kpc, 
#                                                'n_02':7.5e-3*u.cm**-3, 'beta2':0.32, 'r_c2':19.35*u.kpc}
plt.figure(figsize = (12,9))
rad, dens = clust.get_density_gas_profile()
dens_XMM = [8.51e-3, 4.23e-3, 2.78e-3, 2.18e-3, 1.58e-3, 1.42e-3]
rad_XMM = r
ax1 = plt.subplot()
ax1.set_xscale("log", nonposx='clip')
ax1.set_yscale("log", nonposy='clip')
ax1.set(title='A2151CB')
yer = [0.24e-3, 0.11e-3, 0.08e-3, 0.08e-3, 0.08e-3, 0.03e-3 ]
xer = [[15,15,15,15,15,15],[15,15,15,15,15,75]]
ax1.errorbar(rad_XMM, dens_XMM, xerr=xer, yerr=yer, fmt = '.k', label = 'XMM-Newton data')
ax1.loglog(rad,dens, '-', label = 'Pressure profile MINOT')

ax1.set_ylim(bottom= 5e-4, top = 5e-2)
ax1.set_xlim(left = 10, right = 5000)
plt.xlabel('Radius, kpc')
plt.ylabel('Density, 1/cm^3')
plt.legend()
plt.plot()

In [ ]:
clust.print_param()

In [ ]:
dens

In [ ]:
clust.print_param()

In [ ]:
clust_two.print_param()

In [ ]:
cosmo.H

In [ ]:
import astropy.cosmology
import astropy.constants as cst
import astropy.units as u
from scipy.optimize import brentq
import matplotlib.pyplot as plt

#===================================================
#========== get M_delta from R_delta
#===================================================
def Mdelta_to_Rdelta(M_delta, redshift, delta=500, cosmo=astropy.cosmology.Planck15):
    """
    Compute a R_delta, delta being the overdensity wrt the local background
    Parameters
    ----------
    - M_delta (Msun): array
    Outputs
    --------
    - R_delta (kpc): array
    """
    
    # local density
    rho = delta*3.0*cosmo.H(redshift)**2 / 8.0 / np.pi / cst.G

    # Radius
    R_delta = ((3.0*M_delta*u.Msun/4.0/np.pi/rho)**(1.0/3)).to_value('kpc')

    return R_deltav

In [ ]:
cst.G